In [1]:
import os

In [2]:
%pwd

'c:\\datascience End to End Projects\\TV sales prediciton\\TV-sales-prediction\\research'

In [3]:
os.chdir("C:\\datascience End to End Projects\\TV sales prediciton\\TV-sales-prediction")  #  but i would like to work with main ProjectML_with_MLFlow file , so for getting i step back in path inorder to enter the main project file i used this command os.chdir("../")
%pwd

'C:\\datascience End to End Projects\\TV sales prediciton\\TV-sales-prediction'

In [4]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path  # this root_dir,test_data_path,model_path,metric_file_name are the variables takes from the model_evaluation configuration code part of config.yaml file 
    test_data_path: Path
    model_path: Path
    metric_file_name: Path
    target_column: str # this target column takes from schema.yaml file 
    

In [5]:
from TV_sales.constants import *
from TV_sales.utils.common import read_yaml, create_directories, save_json

In [7]:
# this template is same for every stage like data_ingestion, data_validation,data_transformation,model_trainer,model_evaluation

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    # only this part of code get changes her actually iam defining a get_model_evaluation_config
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation# here our model ElasticNet reading all params , Target_column of schema 
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path = config.model_path,
            metric_file_name = config.metric_file_name,
            target_column = schema.name,
           
        )

        return model_evaluation_config # so which ever the varaibles we initilized inside the entity those are all getting returned

In [9]:
import os
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score # here i have imported all the evaluation meterics for evaluating our model peformance
import numpy as np
import joblib

In [10]:
# here i have defined one ModelEvaluation class in which i have defined the ModelEvaluationConfig
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    # here i have defiend the eval_metrics which it evaluates our model performance by calculating the rmse,mae,r2
    def eval_metrics(self,actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2
    

# here i have defined one fucntion called log_into_mlflow which it will mlflow 
    def predict_the_model(self):

        test_data = pd.read_csv(self.config.test_data_path) # here iam loading the test dataset
        model = joblib.load(self.config.model_path) # here iam loading my trained model 

        test_x = test_data.drop([self.config.target_column], axis=1) # here iam dropping my targetcolumn in test_x
        test_y = test_data[[self.config.target_column]] # here iam keeping my target column


        predicted_qualities = model.predict(test_x) # here iam predicting my testing data

        (rmse, mae, r2) = self.eval_metrics(test_y, predicted_qualities) # here it is calculating my metrics of the predicted model
            
        # Saving metrics as local
        scores = {"rmse": rmse, "mae": mae, "r2": r2} # these scores are saving inside my artifacts of model_evaluation as a jason file
        save_json(path=Path(self.config.metric_file_name), data=scores)

            



In [11]:
try:
    config = ConfigurationManager() # here i have initlized my ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config() # her eiam getiing my get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config) # here iam passing this  model_evaluation_config to my ModelEvalaution
    model_evaluation_config.predict_the_model() # here iam starting this log_into_mlflow()
except Exception as e: # if anything goes wrong it throws the exception
    raise e

[2024-02-03 20:44:27,378: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-02-03 20:44:27,419: INFO: common: yaml file: params.yaml loaded successfully]
[2024-02-03 20:44:27,429: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-02-03 20:44:27,432: INFO: common: created directory at: artifacts]
[2024-02-03 20:44:27,434: INFO: common: created directory at: artifacts/model_evaluation]
[2024-02-03 20:44:27,849: INFO: common: json file saved at: artifacts\model_evaluation\metrics.json]
